# Part 4b - Model testing locally
 

In [ ]:
!mkdir model

In [ ]:
!aws s3 cp s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-12-01-09-15-45-087/output/model.tar.gz model/

In [ ]:
!tar -xf model/model.tar.gz -C model/

In [ ]:
!rm -r model/checkpoint*

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./model/')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('./model/').to('cpu').eval()

In [ ]:
import pandas as pd
df_test = pd.read_csv('data/test.csv')
ref_summaries = list(df_test['summary'])
texts = list(df_test['text'])

In [ ]:
texts[0]

In [ ]:
input_ids = tokenizer(texts[0], truncation=True, padding='longest', return_tensors="pt").input_ids.to('cpu')

In [ ]:
output = model.generate(input_ids, max_length=20)
predictions = tokenizer.batch_decode(output, skip_special_tokens=True)

In [ ]:
predictions

In [ ]:
candidate_summaries = []

for i, text in enumerate(texts):
    if i % 100 == 0:
        print(i)
    input_ids = tokenizer(text, truncation=True, padding='longest', return_tensors="pt").input_ids.to('cpu')
    output = model.generate(input_ids, max_length=20)
    predictions = tokenizer.batch_decode(output, skip_special_tokens=True)
    candidate_summaries.append(predictions[0])

In [ ]:
file = open("model-summaries.txt", "w")
for s in candidate_summaries:
    file.write(s + "\n")
file.close()

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
calc_rouge_scores(candidate_summaries, ref_summaries)